In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Just normalization for validation and training data
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

data_dir = '/content/drive/MyDrive/yoga2'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
classes = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
device

device(type='cuda', index=0)

In [8]:
from torch.optim import lr_scheduler
google_net = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
num_ftrs = google_net.fc.in_features
# Here the size of each output sample is set to equal to our classes which are 82 in number.
google_net.fc = nn.Linear(num_ftrs, len(classes))
google_net = google_net.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(google_net.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


  0%|          | 0.00/49.7M [00:00<?, ?B/s]

In [9]:
google_net_pretrained = train_model(google_net, criterion, optimizer, lr_scheduler,
                       num_epochs=10)

Epoch 1/10
----------


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train Loss: 2.8414 Acc: 0.3209


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


val Loss: 1.3994 Acc: 0.5991

Epoch 2/10
----------


...

val Loss: 0.5379 Acc: 0.8708

Epoch 10/10
----------


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train Loss: 0.2142 Acc: 0.9482


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


val Loss: 0.5297 Acc: 0.8762

Training complete in 107m 22s
Best val Acc: 0.876153


In [10]:
path=os.path.join(data_dir,'google_net.pth')
torch.save(google_net_pretrained.state_dict(),path)

In [11]:
def evaluteTop1(model, loader):
    model.eval()
    
    correct = 0
    total = len(loader.dataset)

    for x,y in loader:
        x,y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
            correct += torch.eq(pred, y).sum().float().item()
        #correct += torch.eq(pred, y).sum().item()
    return correct / total

def evaluteTop5(model, loader):
    model.eval()
    correct = 0
    total = len(loader.dataset)
    for x, y in loader:
        x,y = x.to(device),y.to(device)
        with torch.no_grad():
            logits = model(x)
            maxk = max((1,5))
            y_resize = y.view(-1,1)
            _, pred = logits.topk(maxk, 1, True, True)
            correct += torch.eq(pred, y_resize).sum().float().item()
    return correct / total

In [12]:
loader=dataloaders['val']

In [13]:
evaluteTop5(google_net_pretrained, loader)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


0.9667318982387475

In [14]:
evaluteTop1(google_net_pretrained, loader)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


0.87615320100643

In [15]:
nb_classes = len(classes)
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = google_net_pretrained(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
#printing confusion matrix
print(confusion_matrix)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


tensor([[ 10.,   0.,   0.,  ...,   0.,   1.,   0.],
        [  0.,  54.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0., 132.,  ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.,  ...,  26.,   1.,   0.],
        [  0.,   0.,   0.,  ...,   0.,  14.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,  48.]])


In [16]:
#per class accuracy
print(confusion_matrix.diag()/confusion_matrix.sum(1))

tensor([0.6250, 0.9153, 0.9496, 0.8400, 0.9706, 0.7308, 0.8958, 0.9000, 0.9333,
        0.9355, 0.8600, 0.4737, 0.9000, 0.9524, 0.9510, 0.9388, 0.7660, 0.8780,
        0.7708, 0.8462, 0.8529, 0.9038, 0.9286, 0.8929, 0.8800, 0.9804, 0.7647,
        0.8438, 0.5882, 0.8846, 0.9091, 0.9574, 0.9268, 0.7556, 0.9857, 0.9388,
        0.6250, 0.9167, 1.0000, 0.8667, 0.9831, 0.6667, 0.9444, 0.7879, 0.8919,
        0.8571, 0.8800, 0.9630, 0.7778, 0.8947, 0.7333, 0.7297, 0.9107, 0.8713,
        0.8696, 0.9483, 0.8667, 0.9231, 0.9014, 0.6538, 0.7708, 0.8537, 0.8551,
        0.8507, 0.6129, 0.9400, 0.9545, 0.9623, 0.8571, 0.8605, 0.8857, 0.7206,
        0.9333, 0.9123, 0.9333, 0.7143, 0.8889, 0.9706, 0.7857, 0.8966, 0.8750,
        0.8889])


In [17]:
TP = confusion_matrix.diag()
for c in range(nb_classes):
    idx = torch.ones(nb_classes).byte()
    idx[c] = 0
    # all non-class samples classified as non-class
    TN = confusion_matrix[idx.nonzero()[:, None], idx.nonzero()].sum() #conf_matrix[idx[:, None], idx].sum() - conf_matrix[idx, c].sum()
    # all non-class samples classified as class
    FP = confusion_matrix[idx, c].sum()
    # all class samples not classified as class
    FN = confusion_matrix[c, idx].sum()
    
    print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(
        c, TP[c], TN, FP, FN))

Class 0
TP 10.0, TN 3560.0, FP 1.0, FN 6.0
Class 1
TP 54.0, TN 3509.0, FP 9.0, FN 5.0
Class 2
TP 132.0, TN 3433.0, FP 5.0, FN 7.0
Class 3
TP 21.0, TN 3550.0, FP 2.0, FN 4.0
Class 4
TP 66.0, TN 3483.0, FP 26.0, FN 2.0
Class 5
TP 38.0, TN 3510.0, FP 15.0, FN 14.0
Class 6
TP 43.0, TN 3526.0, FP 3.0, FN 5.0
Class 7
TP 9.0, TN 3566.0, FP 1.0, FN 1.0
Class 8
TP 14.0, TN 3561.0, FP 1.0, FN 1.0
Class 9
TP 58.0, TN 3512.0, FP 3.0, FN 4.0
Class 10
TP 43.0, TN 3522.0, FP 5.0, FN 7.0
Class 11
TP 9.0, TN 3554.0, FP 4.0, FN 10.0
Class 12
TP 27.0, TN 3545.0, FP 2.0, FN 3.0
Class 13
TP 20.0, TN 3552.0, FP 4.0, FN 1.0
Class 14
TP 97.0, TN 3467.0, FP 8.0, FN 5.0
Class 15
TP 92.0, TN 3475.0, FP 4.0, FN 6.0
Class 16
TP 36.0, TN 3518.0, FP 12.0, FN 11.0
Class 17
TP 36.0, TN 3535.0, FP 1.0, FN 5.0
Class 18
TP 37.0, TN 3523.0, FP 6.0, FN 11.0
Class 19
TP 33.0, TN 3537.0, FP 1.0, FN 6.0
Class 20
TP 29.0, TN 3541.0, FP 2.0, FN 5.0
Class 21
TP 47.0, TN 3516.0, FP 9.0, FN 5.0
Class 22
TP 65.0, TN 3505.0, FP 2.0,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:30.)
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:30.)
  # Remove the CWD from sys.path while we load stuff.


In [18]:
specificity=TP/(TN+FP)
print('specificity of our model is:',specificity)
sensitivity=TP/(TP+FN)
print('sensitivity of our model is:',sensitivity)

specificity of our model is: tensor([0.0028, 0.0153, 0.0375, 0.0060, 0.0187, 0.0108, 0.0122, 0.0026, 0.0040,
        0.0165, 0.0122, 0.0026, 0.0077, 0.0057, 0.0275, 0.0261, 0.0102, 0.0102,
        0.0105, 0.0094, 0.0082, 0.0133, 0.0185, 0.0071, 0.0125, 0.0142, 0.0074,
        0.0077, 0.0057, 0.0065, 0.0085, 0.0128, 0.0108, 0.0097, 0.0196, 0.0131,
        0.0014, 0.0062, 0.0031, 0.0037, 0.0165, 0.0051, 0.0145, 0.0074, 0.0094,
        0.0085, 0.0062, 0.0074, 0.0119, 0.0048, 0.0031, 0.0077, 0.0145, 0.0250,
        0.0114, 0.0156, 0.0037, 0.0102, 0.0182, 0.0048, 0.0105, 0.0099, 0.0167,
        0.0162, 0.0054, 0.0133, 0.0179, 0.0145, 0.0034, 0.0105, 0.0088, 0.0139,
        0.0079, 0.0148, 0.0079, 0.0071, 0.0136, 0.0187, 0.0094, 0.0074, 0.0040,
        0.0136])
sensitivity of our model is: tensor([0.6250, 0.9000, 0.9565, 0.7778, 0.9167, 0.8636, 0.8776, 0.6000, 0.7000,
        0.9062, 0.8776, 0.6000, 0.8182, 0.7692, 0.9417, 0.9388, 0.8571, 0.8571,
        0.8605, 0.8462, 0.8286, 0.8868, 0.915

In [19]:
model = torch.load(path)
model

OrderedDict([('conv1.conv.weight',
              tensor([[[[ 7.0107e-02,  1.6100e-01,  2.8130e-02,  ..., -1.5376e-02,
                         -1.8627e-02, -4.1673e-02],
                        [ 7.0187e-02,  9.8305e-02, -3.3258e-01,  ...,  1.0905e-01,
                          7.1362e-02, -2.2273e-02],
                        [ 4.3253e-02, -3.1519e-01, -4.6776e-01,  ...,  4.6203e-01,
                          4.3589e-03, -3.5722e-03],
                        ...,
                        [ 1.6098e-04,  2.5616e-01,  4.6640e-01,  ..., -4.5587e-01,
                         -1.3803e-02,  4.9300e-02],
                        [ 3.0722e-02,  1.1309e-01, -1.2467e-01,  ..., -7.9046e-02,
                          9.5450e-02,  7.7464e-02],
                        [-1.3623e-02,  7.4531e-03, -9.5916e-02,  ...,  4.8342e-02,
                          5.2191e-02,  1.7916e-02]],
              
                       [[ 6.0278e-02,  1.0975e-01, -7.2069e-02,  ...,  8.7528e-03,
                          2